## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


### BUSINESS UNDERSTANDING

##### KEY QUESTIONS FOR MOVIE STUDIO STRATEGY
1. Which movie genres generate the highest revenue? By analyzing the total earnings from the different genres, we can determine which types of films are more profitable and likely to attract large audiences.
2. Does movie rating or critic score correlate with higher box office success? using ratings from Rotten Tomatoes and movie database, we can measure whether higher-rated movies earn more revenue 
3. Does a longer runtime affect rating or revenue? By analyzing the runtimes in the dataset, we can identify which runtimes generate higher earnings for different types of movies.
4. Does the release timing affect the earnings? By examining which months and their associated revenue, we can identify the optimal times to launch films for maximum success.

# Movie Studio Market Analysis
## Stakeholder
Head of the New Movie Studio

## Problem Statement
Our company is launching a new movie studio and needs to decide what types of films to produce in order to maximize box office success. However, the company currently lacks historical knowledge about which movie characteristics lead to strong financial performance.

Importing libraries for data loading

In [1]:
import sqlite3
import pandas as pd

Inspecting the dataframe through .head() function

In [2]:
df = pd.read_csv("zippedData/tmdb.movies.csv.gz")
print(df.head())

   Unnamed: 0            genre_ids     id original_language  \
0           0      [12, 14, 10751]  12444                en   
1           1  [14, 12, 16, 10751]  10191                en   
2           2        [12, 28, 878]  10138                en   
3           3      [16, 35, 10751]    862                en   
4           4        [28, 878, 12]  27205                en   

                                 original_title  popularity release_date  \
0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
1                      How to Train Your Dragon      28.734   2010-03-26   
2                                    Iron Man 2      28.515   2010-05-07   
3                                     Toy Story      28.005   1995-11-22   
4                                     Inception      27.920   2010-07-16   

                                          title  vote_average  vote_count  
0  Harry Potter and the Deathly Hallows: Part 1           7.7       10788  
1           

Getting number of rows and columns

In [3]:
df.shape

(26517, 10)

Getting variables that are available

In [4]:
df.columns

Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')

Data types and missing values

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


Checking missing values in each column

In [6]:
df.isnull().sum()

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

Checking for duplicates

In [7]:
df.duplicated().sum()

0

There are no duplicate rows in the dataset indicating that each observation is unique.

Generating summary statistics

In [8]:
df.describe()

,Unnamed: 0,id,popularity,vote_average,vote_count
count,26517.00000,26517.000000,26517.000000,26517.000000,26517.000000
mean,13258.00000,295050.153260,3.130912,5.991281,194.224837
std,7654.94288,153661.615648,4.355229,1.852946,960.961095
min,0.00000,27.000000,0.600000,0.000000,1.000000
25%,6629.00000,157851.000000,0.600000,5.000000,2.000000
50%,13258.00000,309581.000000,1.374000,6.000000,5.000000
75%,19887.00000,419542.000000,3.694000,7.000000,28.000000
max,26516.00000,608444.000000,80.773000,10.000000,22186.000000


Getting genre distribution

In [9]:
df['genre_ids'].value_counts().head()

genre_ids
[99]    3700
[]      2479
[18]    2268
[35]    1660
[27]    1145
Name: count, dtype: int64

Dropping columns that we do not need

In [10]:
df = df.drop(columns=['Unnamed: 0'])

Checking for updated data types in our dataset

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 1.8+ MB


Creating cleaned copy

In [12]:
eda_df = df[
    [
        'id',
        'title',
        'original_language',
        'genre_ids',
        'release_date',
        'popularity',
        'vote_average',
        'vote_count'
    ]
]

Accessing and connecting to IMBD database. Due to the size of the database, a direct connection is established to allow to use part of the database. This removes the github issue of the limit of the size of file uploaded.

In [15]:
import os
print(os.path.abspath('im.db'))



c:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\im.db


In [31]:
import sqlite3
import pandas as pd
import zipfile
import os
# Extracting Data For IMDB and Checking of Size(Loading of Data)
zip_path =  r"C:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\zippedData\im.db.zip"
extract_path = r"C:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\zippedData"
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print(os.path.getsize(
    r"C:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\im.db"
))

0


In [32]:
print(os.listdir(extract_path))


['bom.movie_gross.csv.gz', 'im.db', 'im.db.zip', 'rt.movie_info.tsv.gz', 'rt.reviews.tsv.gz', 'tmdb.movies.csv.gz', 'tn.movie_budgets.csv.gz']


In [33]:
db_path = r"C:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\im.db\zippedData\im.db"

print(os.path.getsize(db_path))


169443328


In [34]:
#Connecting to the Database
db_path = r"C:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\zippedData\im.db.zip"
imdb_conn = sqlite3.connect(db_path)

In [36]:
import os

print(imdb_conn)


In [37]:
db_path = r"C:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\zippedData\im.db.zip"
print(os.path.exists(db_path))
print(db_path)


True
C:\Users\bitut\OneDrive\Desktop\project_phase2\Movie-Studio-EDA-Project\zippedData\im.db.zip


In [38]:
import sqlite3
import pandas as pd

imdb_conn = sqlite3.connect("imdb.db")


In [39]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    imdb_conn
)


,name


In [40]:
pd.read_sql("SELECT * FROM sqlite_master;", imdb_conn)


,type,name,tbl_name,rootpage,sql


In [35]:
# Confirm by listing tables
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    imdb_conn
)




DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table';': file is not a database